In [2]:
import os

In [3]:
%pwd

'c:\\End To End AI Project\\End-to-end-Machine-Learning-project-with-mlflow\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\End To End AI Project\\End-to-end-Machine-Learning-project-with-mlflow'

In [1]:
'''
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/entbappy/End-to-end-Machine-Learning-Project-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="entbappy"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6824692c47a369aa6f9eac5b10041d5c8edbcef0"
'''
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/arsalanazhar2003/End-to-end-Machine-Learning-project-with-mlflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="arsalanazhar2003"
os.environ["MLFLOW_TRACKING_PASSWORD"]= "ddbdda8eff3c518a6b060fc3d7a9cd36bc8c292d"


NameError: name 'os' is not defined

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/arsalanazhar2003/End-to-end-Machine-Learning-project-with-mlflow.mlflow",
           
        )
 
        return model_evaluation_config


In [10]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-08-18 10:27:55,560: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-18 10:27:55,569: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-18 10:27:55,581: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-18 10:27:55,591: INFO: common: created directory at: artifacts]
[2024-08-18 10:27:55,598: INFO: common: created directory at: artifacts/model_evaluation]
[2024-08-18 10:27:56,400: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/08/18 10:28:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 3
Created version '3' of model 'ElasticnetModel'.


In [14]:
#!pip install dagshub

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached graphql_core-3.2.3-py3-none-any.whl.metadata (10 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/236.6 kB ? eta -:--:--
   ------ --------------------------------- 41.0/236.6 kB 1.9 MB/s eta 0:00:01
   ------ --------------------------------- 41.0/236.6 kB 1.9 MB/s eta 0:00:01
   ------ --------------------------------- 41.0/236.6 kB 1.9 MB/s eta 0:00:01
   ------ --------------------------------- 41.0/236.6 kB 1.9 MB/s eta 0:00:01
   ------ --------------------------------- 41.0/236.6 kB 1.9 MB/s eta 0:00:01
   ------ --------------------------------- 41.0/236.6 kB 1.9 MB/s eta 0:00:01
   ------------------- ------------------ 122.9/236.6 kB 288.1 kB/s eta 0:00:01
   ------------------- ------------------ 122.9/236.6 kB 288.1 kB/s 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.2.4 requires httpx>=0.25.0, but you have httpx 0.23.3 which is incompatible.


In [15]:
import dagshub
dagshub.init(repo_owner='arsalanazhar2003', repo_name='End-to-end-Machine-Learning-project-with-mlflow', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\End To End AI Project\End-to-end-Machine-Learning-project-with-mlflow\.conda\Lib\site-packages\rich\live.py:229:
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=5a03062e-e075-43e9-8f2f-7f0f9b23ef97&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a8b9d0b0c255d6337b0540fdac3c10be9de8b7294bbab2e47ab0473071486492




Accessing as arsalanazhar2003

[2024-08-18 10:27:21,408: INFO: helpers: Accessing as arsalanazhar2003]


Initialized MLflow to track repo "arsalanazhar2003/End-to-end-Machine-Learning-project-with-mlflow"

[2024-08-18 10:27:23,530: INFO: helpers: Initialized MLflow to track repo "arsalanazhar2003/End-to-end-Machine-Learning-project-with-mlflow"]


Repository arsalanazhar2003/End-to-end-Machine-Learning-project-with-mlflow initialized!

[2024-08-18 10:27:23,552: INFO: helpers: Repository arsalanazhar2003/End-to-end-Machine-Learning-project-with-mlflow initialized!]
[2024-08-18 10:27:23,892: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /arsalanazhar2003/End-to-end-Machine-Learning-project-with-mlflow.mlflow/api/2.0/mlflow/runs/create]
